<a href="https://colab.research.google.com/github/Sanzid-Priam/Estimating-lower-extremity-joint-angles-during-gait-using-reduced-number-of-sensors-count-/blob/main/ICDIP_Kinematics_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Let`s import all packages that we may need:
import numpy
import tensorflow as tf
import statistics
from numpy import loadtxt
import matplotlib.pyplot as plt
import pandas
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GRU,LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statistics import stdev
import math

from tensorflow.keras.models import load_model



import numpy as np
from scipy.signal import butter,filtfilt

import sys
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph
import seaborn as sns # used for plot interactive graph.
import pandas
import matplotlib.pyplot as plt

## for Deep-learing:
import tensorflow.keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
to_categorical([0, 1, 2, 3], num_classes=4)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.utils import np_utils
import itertools
from tensorflow.keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Flatten
from keras.layers import Bidirectional
#import constraint

from sklearn.model_selection import train_test_split
from keras.regularizers import l2


###  Library for attention layers

import pandas as pd
#import pyarrow.parquet as pq # Used to read the data
import os
import numpy as np
from tensorflow.keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from tensorflow.keras.models import Model
from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from tensorflow.keras import optimizers # Allow us to access the Adam class to modify some parameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold # Used to use Kfold to train our model
from tensorflow.keras.callbacks import * # This object helps the model to train in a smarter way, avoiding overfitting

from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
import statistics
import gc
import h5py

### Early stopping

from tensorflow.keras.callbacks import EarlyStopping


from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# Data Loader

In [ ]:
##IMU, Kin-Kinematics, HoF-Histogram of Features

def data_loader(subject):
  with h5py.File('/content/drive/My Drive/Kinematics Prediction/All_subjects_data_kinetics.h5', 'r') as hf:
    All_subjects = hf['All_subjects']
    Subject = All_subjects[subject]

    HOF=Subject['hof']
    IMU_KIN=Subject['IMU_Kin']

    treadmill_hof = HOF['Treadmill']
    levelground_hof = HOF['Levelground']
    slope_hof = HOF['Slope']
    stair_hof = HOF['Stair']
    round_hof = HOF['Round']
    obstacles_hof = HOF['Obstacles']

    treadmill_IMU_kin = IMU_KIN['Treadmill']
    levelground_IMU_kin = IMU_KIN['Levelground']
    slope_IMU_kin = IMU_KIN['Slope']
    stair_IMU_kin = IMU_KIN['Stair']
    round_IMU_kin= IMU_KIN['Round']
    obstacles_IMU_kin = IMU_KIN['Obstacles']


    hof_data=np.concatenate((treadmill_hof,levelground_hof,slope_hof,stair_hof,round_hof,obstacles_hof),axis=0)
    IMU_kin_data=np.concatenate((treadmill_IMU_kin,levelground_IMU_kin,slope_IMU_kin,stair_IMU_kin,round_IMU_kin,obstacles_IMU_kin),axis=0)

    return np.array(hof_data), np.array(IMU_kin_data)


In [ ]:
subject_1_data_hof, subject_1_data_IMU_Kin=data_loader('Subject_1')
gc.collect()
subject_2_data_hof, subject_2_data_IMU_Kin=data_loader('Subject_2')
gc.collect()
subject_3_data_hof, subject_3_data_IMU_Kin=data_loader('Subject_3')
gc.collect()
subject_4_data_hof, subject_4_data_IMU_Kin=data_loader('Subject_4')
gc.collect()
subject_5_data_hof, subject_5_data_IMU_Kin=data_loader('Subject_5')
gc.collect()
subject_6_data_hof, subject_6_data_IMU_Kin=data_loader('Subject_6')
gc.collect()
subject_7_data_hof, subject_7_data_IMU_Kin=data_loader('Subject_7')
gc.collect()
subject_8_data_hof, subject_8_data_IMU_Kin=data_loader('Subject_8')
gc.collect()
subject_9_data_hof, subject_9_data_IMU_Kin=data_loader('Subject_9')
gc.collect()
subject_10_data_hof, subject_10_data_IMU_Kin=data_loader('Subject_10')
gc.collect()

#Subject Selection

In [ ]:
main_dir = "/content/drive/My Drive/public dataset/Public_dataset_2/Subject01"
os.mkdir(main_dir)
path="/content/"
subject='Subject_01'
encoder='lstm'

In [ ]:
train_data_hof=np.concatenate((subject_1_data_hof,subject_2_data_hof,subject_4_data_hof,subject_5_data_hof,subject_6_data_hof,
                               subject_7_data_hof,subject_8_data_hof,subject_9_data_hof,subject_10_data_hof),axis=0)

train_data_IMU_Kin=np.concatenate((subject_1_data_IMU_Kin,subject_2_data_IMU_Kin,subject_4_data_IMU_Kin,subject_5_data_IMU_Kin,subject_6_data_IMU_Kin,
                               subject_7_data_IMU_Kin,subject_8_data_IMU_Kin,subject_9_data_IMU_Kin,subject_10_data_IMU_Kin),axis=0)

test_data_hof=subject_3_data_hof
test_data_IMU_Kin=subject_3_data_IMU_Kin

# Data Processing

In [ ]:
##### IMUs-0:48
# Sensor 1- Sternum
# Sensor 2-Sacrum
# Sensor 3-R_thigh
# Sensor 4-L_thigh
# Sensor 5-R_shank
# Sensor 6-L_shank
# Sensor 7-R_dorsal
# Sensor 8-L_dorsal

train_dataset_IMU=train_data_IMU_Kin[:,36:48]
train_dataset_hof=train_data_hof
train_dataset_target=np.concatenate((train_data_IMU_Kin[:,55:56],train_data_IMU_Kin[:,58:60],train_data_IMU_Kin[:,62:63],train_data_IMU_Kin[:,65:67]),axis=1) ## Left and right leg hip, knee,ankle angle


test_dataset_IMU=test_data_IMU_Kin[:,36:48]
test_dataset_hof=test_data_hof
test_dataset_target=np.concatenate((test_data_IMU_Kin[:,55:56],test_data_IMU_Kin[:,58:60],test_data_IMU_Kin[:,62:63],test_data_IMU_Kin[:,65:67]),axis=1)

print(train_dataset_IMU.shape)
print(train_dataset_hof.shape)
print(train_dataset_target.shape)


gc.collect()
gc.collect()
gc.collect()


# Data creation

In [ ]:
## Creating a dataset with overlapping window of 100 samples with overlap of 50 samples ##

# # convert an array of values into a dataset matrix
def create_dataset_present(dataset_1, window=100):
  dataX= []
  k=0
  shift=50
  for i in range(int(len(dataset_1)/shift)-1):
    j=shift*k
    a = dataset_1[j:j+window,:]
    dataX.append(a)
    k=k+1
  return np.array(dataX)

In [ ]:
import gc
gc.collect()

In [ ]:
### Reconstruction/Present Dataset ###
w=100

train_X_3=create_dataset_present(train_dataset_IMU,w)
train_y_3=create_dataset_present(train_dataset_target,w)

test_X_1D=create_dataset_present(test_dataset_IMU,w)
test_y=create_dataset_present(test_dataset_target,w)

In [ ]:
train_y_3=train_y_3.reshape(train_y_3.shape[0],w*6)
test_y=test_y.reshape(test_y.shape[0],w*6)


In [ ]:
train_X_1D, X_validation_1D, train_y_5, Y_validation = train_test_split(train_X_3,train_y_3, test_size=0.20, random_state=True)


train_X_1D=train_X_1D
X_validation_1D=X_validation_1D
test_X_1D=test_X_1D

print(train_X_1D.shape,train_y_5.shape,X_validation_1D.shape,Y_validation.shape)

In [ ]:
features=6

train_X_2D=train_X_1D[:,:,0:12].reshape(train_X_1D.shape[0],train_X_1D.shape[1],features,2)
test_X_2D=test_X_1D[:,:,0:12].reshape(test_X_1D.shape[0],test_X_1D.shape[1],features,2)
X_validation_2D= X_validation_1D[:,:,0:12].reshape(X_validation_1D.shape[0],
                                                   X_validation_1D.shape[1],features,2)


print(train_X_2D.shape,test_X_2D.shape,X_validation_2D.shape)

# Different Function for models

In [ ]:
def prediction_test(yhat,test_y_up):

    test_o=test_y_up
    yhat=yhat

    y_1=yhat[:,0]
    y_2=yhat[:,1]
    y_3=yhat[:,2]
    y_4=yhat[:,3]
    y_5=yhat[:,4]
    y_6=yhat[:,5]


    y_test_1=test_o[:,0]
    y_test_2=test_o[:,1]
    y_test_3=test_o[:,2]
    y_test_4=test_o[:,3]
    y_test_5=test_o[:,4]
    y_test_6=test_o[:,5]


    ###calculate RMSE

    rmse_1 =np.sqrt(mean_squared_error(y_test_1,y_1))
    rmse_2 =np.sqrt(mean_squared_error(y_test_2,y_2))
    rmse_3 =np.sqrt(mean_squared_error(y_test_3,y_3))
    rmse_4 =np.sqrt(mean_squared_error(y_test_4,y_4))
    rmse_5 =np.sqrt(mean_squared_error(y_test_5,y_5))
    rmse_6 =np.sqrt(mean_squared_error(y_test_6,y_6))


    p_1=np.corrcoef(y_1, y_test_1)[0, 1]
    p_2=np.corrcoef(y_2, y_test_2)[0, 1]
    p_3=np.corrcoef(y_3, y_test_3)[0, 1]
    p_4=np.corrcoef(y_4, y_test_4)[0, 1]
    p_5=np.corrcoef(y_5, y_test_5)[0, 1]
    p_6=np.corrcoef(y_6, y_test_6)[0, 1]

    ### Getiing single RMSE and PCC value for a joint
    p=np.array([(p_1+p_4)/2,(p_2+p_5)/2,(p_3+p_6)/2])

    rmse=np.array([(rmse_1+rmse_4)/2,(rmse_2+rmse_5)/2,(rmse_3+rmse_6)/2])



    return rmse,p

In [ ]:
## Evaluting data with the original form without any overlaps

# # convert an array of values into a dataset matrix
def unpack_dataset_present(dataset_1):
  dataX= []
  k=1
  l=0
  shift=100
  for i in range(int(len(dataset_1)/shift)-1):
    j=shift*k
    a = dataset_1[l:j,:]
    l=0
    l=j+50
    dataX=np.append(dataX,a)
    k=k+1
    j=0
  return np.array(dataX)

# Loss Function

In [ ]:
def correlation_coefficient_loss_RMSE(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    l1=K.sqrt(K.mean(K.square(y - x)))

    return l1


In [ ]:
def correlation_coefficient_loss_joint(y_true, y_pred):
    # Calculate mean values
    mx = tf.reduce_mean(y_true)
    my = tf.reduce_mean(y_pred)

    # Calculate differences from mean
    xm, ym = y_true - mx, y_pred - my

    # Calculate numerator and denominator of Pearson correlation coefficient
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.multiply(tf.reduce_sum(tf.square(xm)), tf.reduce_sum(tf.square(ym))))

    # Calculate Pearson correlation coefficient
    r = r_num / r_den

    # Clamp r between -1 and 1
    r = tf.maximum(tf.minimum(r, 1.0), -1.0)

    # Calculate l2 loss
    l2 = 1 - tf.square(r)

    l1=K.sqrt(K.mean(K.square(y_true - y_pred)))


    return l1+l2

# Model Ablation

##1. GRU-Net

In [ ]:
def GRU_Net(inputs_1D_N,inputs_2D_N):

  model_1=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_1=Dropout(0.4)(model_1)
  model_1=Bidirectional(GRU(64,return_sequences=True))(model_1)
  model_1=Dropout(0.4)(model_1)
  model_1=Flatten()(model_1)

  output_GRU=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(model_1)


  return output_GRU

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output=GRU_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=output)
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], train_y_5, epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], Y_validation), verbose=2, shuffle=False)

filename = path+'model_GRU.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_GRU.h5'

model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

yhat_4=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_GRU=rmse
PCC_GRU=p

Ablation_1=np.hstack([RMSE_GRU,PCC_GRU])

## 2. Conv1D-Net

In [ ]:
def Conv1D_Net(inputs_1D_N,inputs_2D_N):

  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(inputs_1D_N)
  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)

  CNN=Dense(64, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)
  CNN=Dense(32, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)

  CNN=Flatten()(CNN)

  output_C1=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(CNN)


  return output_C1

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )

inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)

output=Conv1D_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=output)
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], train_y_5, epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], Y_validation), verbose=2, shuffle=False)

filename = path+'model_Conv1D.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Conv1D.h5'

model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

yhat_4=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Conv1D=rmse
PCC_Conv1D=p

Ablation_2=np.hstack([RMSE_Conv1D,PCC_Conv1D])

## 3. Conv2D-Net

In [ ]:
def Conv2D_Net(inputs_1D_N,inputs_2D_N):

  X=Conv2D(64, (3, 3), activation='relu',padding='same')(inputs_2D_N)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Conv2D(128, (3, 3), activation='relu', padding='same')(X)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Dense(64, activation='relu')(X)

  X=Dropout(0.2)(X)
  X=Dense(32,activation='relu')(X)
  X=Dropout(0.2)(X)

  X=Flatten()(X)
  output_C2=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(X)


  return output_C2

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output=Conv2D_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=output)
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], train_y_5, epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], Y_validation), verbose=2, shuffle=False)

filename = path+'model_Conv2D.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Conv2D.h5'

model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

yhat_4=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Conv2D=rmse
PCC_Conv2D=p

Ablation_3=np.hstack([RMSE_Conv2D,PCC_Conv2D])

##4. GRU-Conv1D-Net

In [ ]:
def GRU_Conv1D_Net(inputs_1D_N,inputs_2D_N):

  model_3=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_3=Dropout(0.4)(model_3)
  model_3=Bidirectional(GRU(64,return_sequences=True))(model_3)
  model_3=Dropout(0.4)(model_3)
  model_3=Flatten()(model_3)

  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(inputs_1D_N)
  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)


  CNN=Dense(64, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)
  CNN=Dense(32, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)

  CNN_1=Flatten()(CNN)
  CNN= concatenate([model_3,CNN_1])

  output_C1=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(CNN)


  return output_C1

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )

inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)

output=GRU_Conv1D_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=output)
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], train_y_5, epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], Y_validation), verbose=2, shuffle=False)

filename = path+'model_GRU_Conv1D.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_GRU_Conv1D.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})


yhat_4=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_GRU_Conv1D=rmse
PCC_GRU_Conv1D=p

Ablation_4=np.hstack([RMSE_GRU_Conv1D,PCC_GRU_Conv1D])

##5. GRU-Conv2D-Net

In [ ]:
def GRU_Conv2D_Net(inputs_1D_N,inputs_2D_N):

  model_2=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_2=Dropout(0.4)(model_2)
  model_2=Bidirectional(GRU(64,return_sequences=True))(model_2)
  model_2=Dropout(0.4)(model_2)
  model_2=Flatten()(model_2)

  X=Conv2D(64, (3, 3), activation='relu',padding='same')(inputs_2D_N)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Conv2D(128, (3, 3), activation='relu', padding='same')(X)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)

  X=Dense(64, activation='relu')(X)
  X=Dropout(0.2)(X)
  X=Dense(32,activation='relu')(X)
  X=Dropout(0.2)(X)

  X_1=Flatten()(X)
  X= concatenate([model_2,X_1])

  output_C2=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(X)


  return output_C2

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output=GRU_Conv2D_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=output)
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], train_y_5, epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], Y_validation), verbose=2, shuffle=False)

filename = path+'model_GRU_Conv2D.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_GRU_Conv2D.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

yhat_4=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_GRU_Conv2D=rmse
PCC_GRU_Conv2D=p

Ablation_5=np.hstack([RMSE_GRU_Conv2D,PCC_GRU_Conv2D])

##6. Conv1D-Conv2D-Net

In [ ]:
def Conv1D_Conv2D_Net(inputs_1D_N,inputs_2D_N):

  X=Conv2D(64, (3, 3), activation='relu',padding='same')(inputs_2D_N)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Conv2D(128, (3, 3), activation='relu', padding='same')(X)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)

  X=Dense(64, activation='relu')(X)
  X=Dropout(0.2)(X)
  X=Dense(32,activation='relu')(X)
  X=Dropout(0.2)(X)
  X_1=Flatten()(X)

  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(inputs_1D_N)
  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)
  CNN=Dense(64, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)
  CNN=Dense(32, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)
  CNN_1=Flatten()(CNN)

  CNN= concatenate([X_1,CNN_1])
  output_C1=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(CNN)


  return output_C1

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output=Conv1D_Conv2D_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=output)
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], train_y_5, epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], Y_validation), verbose=2, shuffle=False)

filename = path+'model_Conv1D_Conv2D.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Conv1D_Conv2D.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

yhat_4=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Conv1D_Conv2D=rmse
PCC_Conv1D_Conv2D=p

Ablation_6=np.hstack([RMSE_Conv1D_Conv2D,PCC_Conv1D_Conv2D])

##7. Gait-SubNet-1

In [ ]:
def Gait_SubNet_1(inputs_1D_N,inputs_2D_N):

  model_1=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_1=Dropout(0.4)(model_1)
  model_1=Bidirectional(GRU(64,return_sequences=True))(model_1)
  model_1=Dropout(0.4)(model_1)
  model_1=Flatten()(model_1)

  model_3=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_3=Dropout(0.4)(model_3)
  model_3=Bidirectional(GRU(64,return_sequences=True))(model_3)
  model_3=Dropout(0.4)(model_3)
  model_3=Flatten()(model_3)

  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(inputs_1D_N)
  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)

  CNN=Dense(64, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)
  CNN=Dense(32, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)
  CNN_1=Flatten()(CNN)
  CNN= concatenate([model_3,CNN_1])

  output_GRU=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(model_1)
  output_C1=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(CNN)

  output = Average()([output_GRU,output_C1])


  return (output_GRU,output_C1,output)

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output=Gait_SubNet_1(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output])
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_subNet_1.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_subNet_1.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

yhat_1,yhat_2,yhat_4=model.predict([test_X_1D,test_X_2D])

### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_subNet_1=rmse
PCC_Gait_subNet_1=p

Ablation_7=np.hstack([RMSE_Gait_subNet_1,PCC_Gait_subNet_1])

##8. Gait-SubNet-2

In [ ]:
def Gait_SubNet_2(inputs_1D_N,inputs_2D_N):

  model_1=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_1=Dropout(0.4)(model_1)
  model_1=Bidirectional(GRU(64,return_sequences=True))(model_1)
  model_1=Dropout(0.4)(model_1)
  model_1=Flatten()(model_1)

  model_2=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_2=Dropout(0.4)(model_2)
  model_2=Bidirectional(GRU(64,return_sequences=True))(model_2)
  model_2=Dropout(0.4)(model_2)
  model_2=Flatten()(model_2)

  X=Conv2D(64, (3, 3), activation='relu',padding='same')(inputs_2D_N)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Conv2D(128, (3, 3), activation='relu', padding='same')(X)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Dense(64, activation='relu')(X)
  X=Dropout(0.2)(X)
  X=Dense(32,activation='relu')(X)
  X=Dropout(0.2)(X)

  X_1=Flatten()(X)
  X= concatenate([model_2,X_1])

  output_GRU=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(model_1)
  output_C2=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(X)

  output = Average()([output_GRU,output_C2])

  return (output_GRU,output_C2,output)

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output=Gait_SubNet_2(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output])
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_subNet_2.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_subNet_2.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

yhat_1,yhat_2,yhat_4=model.predict([test_X_1D,test_X_2D])

### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_subNet_2=rmse
PCC_Gait_subNet_2=p

Ablation_8=np.hstack([RMSE_Gait_subNet_2,PCC_Gait_subNet_2])

##9. Gait-SubNet-3

In [ ]:
def Gait_SubNet_3(inputs_1D_N,inputs_2D_N):

  model_2=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_2=Dropout(0.4)(model_2)
  model_2=Bidirectional(GRU(64,return_sequences=True))(model_2)
  model_2=Dropout(0.4)(model_2)
  model_2=Flatten()(model_2)

  X=Conv2D(64, (3, 3), activation='relu',padding='same')(inputs_2D_N)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Conv2D(128, (3, 3), activation='relu', padding='same')(X)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)

  X=Dense(64, activation='relu')(X)
  X=Dropout(0.2)(X)
  X=Dense(32,activation='relu')(X)
  X=Dropout(0.2)(X)
  X_1=Flatten()(X)
  X= concatenate([model_2,X_1])

  model_3=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_3=Dropout(0.4)(model_3)
  model_3=Bidirectional(GRU(64,return_sequences=True))(model_3)
  model_3=Dropout(0.4)(model_3)
  model_3=Flatten()(model_3)

  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(inputs_1D_N)
  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)

  CNN=Dense(64, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)
  CNN=Dense(32, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)

  CNN_1=Flatten()(CNN)


  CNN= concatenate([model_3,CNN_1])

  output_C2=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(X)
  output_C1=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(CNN)

  output = Average()([output_C2,output_C1])


  return (output_C1,output_C2,output)

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input(shape=(w,12))
inputs_2D = tf.keras.layers.Input(shape=(w,6,2))

inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)

output_1,output_2,output=Gait_SubNet_3(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output])
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_subNet_3.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_subNet_3.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})


yhat_1,yhat_2,yhat_4=model.predict([test_X_1D,test_X_2D])

### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_subNet_3=rmse
PCC_Gait_subNet_3=p

Ablation_9=np.hstack([RMSE_Gait_subNet_3,PCC_Gait_subNet_3])

##10. Gait-Net

In [ ]:
def Gait_Net(inputs_1D_N,inputs_2D_N):

  model_1=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_1=Dropout(0.4)(model_1)
  model_1=Bidirectional(GRU(64,return_sequences=True))(model_1)
  model_1=Dropout(0.4)(model_1)
  model_1=Flatten()(model_1)

  model_2=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_2=Dropout(0.4)(model_2)
  model_2=Bidirectional(GRU(64,return_sequences=True))(model_2)
  model_2=Dropout(0.4)(model_2)
  model_2=Flatten()(model_2)

  X=Conv2D(64, (3, 3), activation='relu',padding='same')(inputs_2D_N)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Conv2D(128, (3, 3), activation='relu', padding='same')(X)
  X=BatchNormalization()(X)
  X=MaxPooling2D((2, 2))(X)
  X=Dense(64, activation='relu')(X)
  X=Dropout(0.2)(X)
  X=Dense(32,activation='relu')(X)
  X=Dropout(0.2)(X)

  X_1=Flatten()(X)
  X= concatenate([model_2,X_1])


  model_3=Bidirectional(GRU(128,return_sequences=True))(inputs_1D_N)
  model_3=Dropout(0.4)(model_3)
  model_3=Bidirectional(GRU(64,return_sequences=True))(model_3)
  model_3=Dropout(0.4)(model_3)
  model_3=Flatten()(model_3)

  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(inputs_1D_N)
  CNN=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=Conv1D(filters=128, kernel_size=3, activation='relu',padding='same')(CNN)
  CNN=BatchNormalization()(CNN)
  CNN=MaxPooling1D(pool_size=2)(CNN)


  CNN=Dense(64, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)
  CNN=Dense(32, activation='relu')(CNN)
  CNN=Dropout(0.2)(CNN)

  CNN_1=Flatten()(CNN)


  CNN= concatenate([model_3,CNN_1])

  output_GRU=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(model_1)
  output_C2=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(X)
  output_C1=Dense(6*w,bias_regularizer=l2(0.001), activation='linear')(CNN)


  output = Average()([output_GRU,output_C2,output_C1])

  return (output_GRU,output_C2,output_C1,output)

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )

inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)

output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_Net.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_Net.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_Net=rmse
PCC_Gait_Net=p

Ablation_10=np.hstack([RMSE_Gait_Net,PCC_Gait_Net])

## 11. Gait-Net--Joint Loss

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_joint, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_JL_Net.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_GRU_Conv2D.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5),80)
test_y_up=unpack_dataset_present(np.array(test_y_r),80)

print(yhat_up.shape,test_y_up.shape)


### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_JL_Net=rmse
PCC_Gait_JL_Net=p

Ablation_11=np.hstack([RMSE_Gait_JL_Net,PCC_Gait_JL_Net])

# Augmentation+ Joint Loss analysis

## Data Loader

In [ ]:
def data_loader(subject):
  with h5py.File('/content/drive/My Drive/Kinematics Prediction/All_subjects_data_kinetics.h5', 'r') as hf:
    All_subjects = hf['All_subjects']
    Subject = All_subjects[subject]

    HOF=Subject['hof']
    IMU_KIN=Subject['IMU_Kin']

    treadmill_hof = HOF['Treadmill']
    levelground_hof = HOF['Levelground']
    slope_hof = HOF['Slope']
    stair_hof = HOF['Stair']
    round_hof = HOF['Round']
    obstacles_hof = HOF['Obstacles']

    treadmill_IMU_kin = IMU_KIN['Treadmill']
    levelground_IMU_kin = IMU_KIN['Levelground']
    slope_IMU_kin = IMU_KIN['Slope']
    stair_IMU_kin = IMU_KIN['Stair']
    round_IMU_kin= IMU_KIN['Round']
    obstacles_IMU_kin = IMU_KIN['Obstacles']


    hof_data=np.concatenate((treadmill_hof,levelground_hof,levelground_hof,slope_hof,slope_hof,slope_hof,stair_hof,stair_hof,stair_hof,stair_hof,round_hof,obstacles_hof,round_hof,\
                             obstacles_hof,round_hof,obstacles_hof,round_hof,obstacles_hof),axis=0)
    IMU_kin_data=np.concatenate((treadmill_IMU_kin,levelground_IMU_kin,levelground_IMU_kin,slope_IMU_kin,slope_IMU_kin,slope_IMU_kin,stair_IMU_kin,stair_IMU_kin,\
                                 stair_IMU_kin,round_IMU_kin,obstacles_IMU_kin,round_IMU_kin,obstacles_IMU_kin,round_IMU_kin,obstacles_IMU_kin,round_IMU_kin,obstacles_IMU_kin),axis=0)

    return np.array(hof_data), np.array(IMU_kin_data)


In [ ]:
subject_1_data_hof_aug, subject_1_data_IMU_Kin_aug=data_loader('Subject_1')
gc.collect()
subject_2_data_hof_aug, subject_2_data_IMU_Kin_aug=data_loader('Subject_2')
gc.collect()
subject_3_data_hof_aug, subject_3_data_IMU_Kin_aug=data_loader('Subject_3')
gc.collect()
subject_4_data_hof_aug, subject_4_data_IMU_Kin_aug=data_loader('Subject_4')
gc.collect()
subject_5_data_hof_aug, subject_5_data_IMU_Kin_aug=data_loader('Subject_5')
gc.collect()
subject_6_data_hof_aug, subject_6_data_IMU_Kin_aug=data_loader('Subject_6')
gc.collect()
subject_7_data_hof_aug, subject_7_data_IMU_Kin_aug=data_loader('Subject_7')
gc.collect()
subject_8_data_hof_aug, subject_8_data_IMU_Kin_aug=data_loader('Subject_8')
gc.collect()
subject_9_data_hof_aug, subject_9_data_IMU_Kin_aug=data_loader('Subject_9')
gc.collect()
subject_10_data_hof_aug, subject_10_data_IMU_Kin_aug=data_loader('Subject_10')
gc.collect()

##Subject Selection

In [ ]:
main_dir = "/content/drive/My Drive/public dataset/Public_dataset_2/Subject01"
# os.mkdir(main_dir)
path="/content/"
subject='Subject_01'
encoder='lstm'

In [ ]:
train_data_hof_aug=np.concatenate((subject_1_data_hof_aug,subject_2_data_hof_aug,subject_4_data_hof_aug,subject_5_data_hof_aug,subject_6_data_hof_aug,
                               subject_7_data_hof_aug,subject_8_data_hof_aug,subject_9_data_hof_aug,subject_10_data_hof_aug),axis=0)

train_data_IMU_Kin_aug=np.concatenate((subject_1_data_IMU_Kin_aug,subject_2_data_IMU_Kin_aug,subject_4_data_IMU_Kin_aug,subject_5_data_IMU_Kin_aug,subject_6_data_IMU_Kin_aug,
                               subject_7_data_IMU_Kin_aug,subject_8_data_IMU_Kin_aug,subject_9_data_IMU_Kin_aug,subject_10_data_IMU_Kin_aug),axis=0)

test_data_hof=subject_3_data_hof
test_data_IMU_Kin=subject_3_data_IMU_Kin

## Data Processing

In [ ]:
##### IMUs-0:48
# Sensor 1- Sternum
# Sensor 2-Sacrum
# Sensor 3-R_thigh
# Sensor 4-L_thigh
# Sensor 5-R_shank
# Sensor 6-L_shank
# Sensor 7-R_dorsal
# Sensor 8-L_dorsal

train_dataset_IMU=train_data_IMU_Kin_aug[:,36:48]
train_dataset_hof=train_data_hof_aug
train_dataset_target=np.concatenate((train_data_IMU_Kin_aug[:,55:56],train_data_IMU_Kin_aug[:,58:60],train_data_IMU_Kin_aug[:,62:63],train_data_IMU_Kin_aug[:,65:67]),axis=1)


test_dataset_IMU=test_data_IMU_Kin[:,36:48]
test_dataset_hof=test_data_hof
test_dataset_target=np.concatenate((test_data_IMU_Kin[:,55:56],test_data_IMU_Kin[:,58:60],test_data_IMU_Kin[:,62:63],test_data_IMU_Kin[:,65:67]),axis=1)

print(train_dataset_IMU.shape)
print(train_dataset_hof.shape)
print(train_dataset_target.shape)


gc.collect()
gc.collect()
gc.collect()


## Data creation

In [ ]:
# # convert an array of values into a dataset matrix
def create_dataset_present(dataset_1, window=100):
  dataX= []
  k=0
  shift=50
  for i in range(int(len(dataset_1)/shift)-1):
    j=shift*k
    a = dataset_1[j:j+window,:]
    dataX.append(a)
    k=k+1
  return np.array(dataX)

In [ ]:
import gc
gc.collect()

In [ ]:
### Reconstruction/Present Dataset ###
w=100

train_X_3=create_dataset_present(train_dataset_IMU,w)
train_y_3=create_dataset_present(train_dataset_target,w)

test_X_1D=create_dataset_present(test_dataset_IMU,w)
test_y=create_dataset_present(test_dataset_target,w)

In [ ]:
train_y_3=train_y_3.reshape(train_y_3.shape[0],w*6)
test_y=test_y.reshape(test_y.shape[0],w*6)


In [ ]:
train_X_1D, X_validation_1D, train_y_5, Y_validation = train_test_split(train_X_3,train_y_3, test_size=0.20, random_state=True)


train_X_1D=train_X_1D
X_validation_1D=X_validation_1D
test_X_1D=test_X_1D

print(train_X_1D.shape,train_y_5.shape,X_validation_1D.shape,Y_validation.shape)

In [ ]:
features=6

train_X_2D=train_X_1D[:,:,0:12].reshape(train_X_1D.shape[0],train_X_1D.shape[1],features,2)
test_X_2D=test_X_1D[:,:,0:12].reshape(test_X_1D.shape[0],test_X_1D.shape[1],features,2)
X_validation_2D= X_validation_1D[:,:,0:12].reshape(X_validation_1D.shape[0],
                                                   X_validation_1D.shape[1],features,2)


print(train_X_2D.shape,test_X_2D.shape,X_validation_2D.shape)

## 12. Gait-Net+Augmentation

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input(shape=(w,12))
inputs_2D = tf.keras.layers.Input(shape=(w,6,2))

inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)

output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_RMSE, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_A_Net.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_A_Net.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_RMSE': correlation_coefficient_loss_RMSE})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)
### Future ###

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)


### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_A_Net=rmse
PCC_Gait_A_Net=p

Ablation_12=np.hstack([RMSE_Gait_A_Net,PCC_Gait_A_Net])

## 13. Gait-JL-Net+Augmentation

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_joint, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_JL_A_Net.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_JL_A_Net.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_joint': correlation_coefficient_loss_joint})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)


### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_JL_A_Net=rmse
PCC_Gait_JL_A_Net=p

Ablation_13=np.hstack([RMSE_Gait_JL_A_Net,PCC_Gait_JL_A_Net])

## Loss function with respect to weight

In [ ]:
def correlation_coefficient_loss_joint_2(y_true, y_pred):
    # Calculate mean values
    mx = tf.reduce_mean(y_true)
    my = tf.reduce_mean(y_pred)

    # Calculate differences from mean
    xm, ym = y_true - mx, y_pred - my

    # Calculate numerator and denominator of Pearson correlation coefficient
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.multiply(tf.reduce_sum(tf.square(xm)), tf.reduce_sum(tf.square(ym))))

    # Calculate Pearson correlation coefficient
    r = r_num / r_den

    # Clamp r between -1 and 1
    r = tf.maximum(tf.minimum(r, 1.0), -1.0)

    # Calculate l2 loss
    l2 = 1 - tf.square(r)

    l1=K.sqrt(K.mean(K.square(y_true - y_pred)))


    return l1+2*l2

In [ ]:
def correlation_coefficient_loss_joint_4(y_true, y_pred):
    # Calculate mean values
    mx = tf.reduce_mean(y_true)
    my = tf.reduce_mean(y_pred)

    # Calculate differences from mean
    xm, ym = y_true - mx, y_pred - my

    # Calculate numerator and denominator of Pearson correlation coefficient
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.multiply(tf.reduce_sum(tf.square(xm)), tf.reduce_sum(tf.square(ym))))

    # Calculate Pearson correlation coefficient
    r = r_num / r_den

    # Clamp r between -1 and 1
    r = tf.maximum(tf.minimum(r, 1.0), -1.0)

    # Calculate l2 loss
    l2 = 1 - tf.square(r)

    l1=K.sqrt(K.mean(K.square(y_true - y_pred)))


    return l1+4*l2

In [ ]:
def correlation_coefficient_loss_joint_6(y_true, y_pred):
    # Calculate mean values
    mx = tf.reduce_mean(y_true)
    my = tf.reduce_mean(y_pred)

    # Calculate differences from mean
    xm, ym = y_true - mx, y_pred - my

    # Calculate numerator and denominator of Pearson correlation coefficient
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.multiply(tf.reduce_sum(tf.square(xm)), tf.reduce_sum(tf.square(ym))))

    # Calculate Pearson correlation coefficient
    r = r_num / r_den

    # Clamp r between -1 and 1
    r = tf.maximum(tf.minimum(r, 1.0), -1.0)

    # Calculate l2 loss
    l2 = 1 - tf.square(r)

    l1=K.sqrt(K.mean(K.square(y_true - y_pred)))


    return l1+6*l2

In [ ]:
def correlation_coefficient_loss_joint_8(y_true, y_pred):
    # Calculate mean values
    mx = tf.reduce_mean(y_true)
    my = tf.reduce_mean(y_pred)

    # Calculate differences from mean
    xm, ym = y_true - mx, y_pred - my

    # Calculate numerator and denominator of Pearson correlation coefficient
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.multiply(tf.reduce_sum(tf.square(xm)), tf.reduce_sum(tf.square(ym))))

    # Calculate Pearson correlation coefficient
    r = r_num / r_den

    # Clamp r between -1 and 1
    r = tf.maximum(tf.minimum(r, 1.0), -1.0)

    # Calculate l2 loss
    l2 = 1 - tf.square(r)

    l1=K.sqrt(K.mean(K.square(y_true - y_pred)))


    return l1+8*l2

In [ ]:
def correlation_coefficient_loss_joint_10(y_true, y_pred):
    # Calculate mean values
    mx = tf.reduce_mean(y_true)
    my = tf.reduce_mean(y_pred)

    # Calculate differences from mean
    xm, ym = y_true - mx, y_pred - my

    # Calculate numerator and denominator of Pearson correlation coefficient
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.multiply(tf.reduce_sum(tf.square(xm)), tf.reduce_sum(tf.square(ym))))

    # Calculate Pearson correlation coefficient
    r = r_num / r_den

    # Clamp r between -1 and 1
    r = tf.maximum(tf.minimum(r, 1.0), -1.0)

    # Calculate l2 loss
    l2 = 1 - tf.square(r)

    l1=K.sqrt(K.mean(K.square(y_true - y_pred)))


    return l1+10*l2

## 14. Gait-JL-Net_2+Augmentation

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_joint_2, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_JL_A_Net_2.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_JL_A_Net_2.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_joint_2': correlation_coefficient_loss_joint_2})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])



 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)


### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_JL_A_Net_2=rmse
PCC_Gait_JL_A_Net_2=p

Ablation_14=np.hstack([RMSE_Gait_JL_A_Net_2,PCC_Gait_JL_A_Net_2])

## 15. Gait-JL-Net_4+Augmentation

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_joint_4, optimizer='Adam')

history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_JL_A_Net_4.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_JL_A_Net_4.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_joint_4': correlation_coefficient_loss_joint_4})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)


### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_JL_A_Net_4=rmse
PCC_Gait_JL_A_Net_4=p

Ablation_15=np.hstack([RMSE_Gait_JL_A_Net_4,PCC_Gait_JL_A_Net_4])

## 16. Gait-JL-Net_6+Augmentation

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_joint_6, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_JL_A_Net_6.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_JL_A_Net_6.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_joint_6': correlation_coefficient_loss_joint_6})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_JL_A_Net_6=rmse
PCC_Gait_JL_A_Net_6=p

Ablation_16=np.hstack([RMSE_Gait_JL_A_Net_6,PCC_Gait_JL_A_Net_6])

## 17. Gait-JL-Net_8+Augmentation

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_joint_8, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_JL_A_Net_8.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_JL_A_Net_8.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_joint_8': correlation_coefficient_loss_joint_8})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)

### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))

print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_JL_A_Net_8=rmse
PCC_Gait_JL_A_Net_8=p

Ablation_17=np.hstack([RMSE_Gait_JL_A_Net_8,PCC_Gait_JL_A_Net_8])

## 18. Gait-JL-Net_10+Augmentation

In [ ]:
#### Main Model ####

inputs_1D = tf.keras.layers.Input( shape=(w,12) )
inputs_2D = tf.keras.layers.Input( shape=(w,6,2) )


inputs_1D_N=BatchNormalization()(inputs_1D)
inputs_2D_N=BatchNormalization()(inputs_2D)


output_1,output_2,output_3,output=Gait_Net(inputs_1D_N,inputs_2D_N)

model = Model(inputs=[inputs_1D, inputs_2D], outputs=[output_1,output_2,output_3,output])
model.compile(loss=correlation_coefficient_loss_joint_10, optimizer='Adam')


history=model.fit([np.array(train_X_1D),np.array(train_X_2D)], [train_y_5,train_y_5,train_y_5,train_y_5], epochs=30, batch_size=64, validation_data=([X_validation_1D,\
                                                                      X_validation_2D], [Y_validation,Y_validation,Y_validation,Y_validation]), verbose=2, shuffle=False)

filename = path+'model_Gait_JL_A_Net_10.h5'
model.save(filename)
print('>Saved %s' % filename)

# # # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
model.summary()


In [ ]:
gc.collect()
gc.collect()
gc.collect()
gc.collect()

In [ ]:
model_path = path+'model_Gait_JL_A_Net_10.h5'
model=load_model(model_path, custom_objects={'correlation_coefficient_loss_joint_10': correlation_coefficient_loss_joint_10})

[yhat_1,yhat_2,yhat_3,yhat_4]=model.predict([test_X_1D,test_X_2D])


 ### Present ###
yhat_5=yhat_4.reshape((yhat_4.shape[0]*w,6))
test_y_r=test_y.reshape((test_y.shape[0]*w,6))

print(yhat_4.shape)


### Unpack ###
yhat_up=unpack_dataset_present(np.array(yhat_5))
test_y_up=unpack_dataset_present(np.array(test_y_r))


print(yhat_up.shape,test_y_up.shape)

### Present ###

yhat_up=yhat_up.reshape(int(len(yhat_up)/6),6)
test_y_up=test_y_up.reshape(int(len(test_y_up)/6),6)

print(yhat_up.shape,test_y_up.shape)

In [ ]:
### Present ###

rmse,p= prediction_test(np.array(yhat_up),np.array(test_y_up))

print(rmse[0])
print(rmse[1])
print(rmse[2])

m=np.mean(rmse)

print('\n')
print(m)

print('\n')

print(p[0])
print(p[1])
print(p[2])
print('\n')

print(np.mean(p))

In [ ]:
RMSE_Gait_JL_A_Net_10=rmse
PCC_Gait_JL_A_Net_10=p

Ablation_18=np.hstack([RMSE_Gait_JL_A_Net_10,PCC_Gait_JL_A_Net_10])

# Results

In [ ]:
all_results=np.vstack([Ablation_1,Ablation_2,Ablation_3,Ablation_4,Ablation_5,Ablation_6,Ablation_7,Ablation_8,Ablation_9,Ablation_10,Ablation_11,Ablation_12,Ablation_13,Ablation_14,Ablation_15,Ablation_16,Ablation_17,Ablation_18])
from numpy import savetxt

savetxt('/content/drive/My Drive/Conference paper/subject 2/subject_2_results.csv', all_results, delimiter=',')